<a href="https://colab.research.google.com/github/Randasabag/Projet-5-ML/blob/main/Copie_de_P5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Partie 1 : Librairies et Données

In [13]:
import numpy
print(numpy.__version__)

import gensim
print(gensim.__version__)

1.26.4
4.3.3


In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud 
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim import corpora
from gensim.models import LdaModel, Word2Vec
from gensim import corpora
import pyLDAvis
import pyLDAvis.gensim
import pyLDAvis.gensim_models
from gensim.models.coherencemodel import CoherenceModel
import pyLDAvis.gensim as gensimvis
import gensim.downloader as api
import warnings
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

# Partie 2 : Méthode non supervisée

In [15]:
df =  pd.read_csv("data_titres.csv")
print(df.shape)

(50000, 11)


In [16]:
df['length_bow'] = df['sentence_bow'].apply(lambda x : len(word_tokenize(x)))
print("max length bow : ", df['length_bow'].max())
df['length_bow_lem'] = df['sentence_bow_lem'].apply(lambda x : len(word_tokenize(x)))
print("max length bow lem : ", df['length_bow_lem'].max())
df['length_dl'] = df['sentence_dl'].apply(lambda x : len(word_tokenize(x)))
print("max length dl : ", df['length_dl'].max())

max length bow :  19
max length bow lem :  19
max length dl :  47


In [17]:
df = df.sample(5000)

In [18]:
df.to_csv("dataset_p5.csv")

## LDA

Latent Dirichlet Allocation : méthode non-supervisée générative

In [19]:
# Creating document-term matrix 
text_list = df['sentence_bow_lem'].tolist()
print(text_list)
# Créer une liste de listes de mots (tokens) à partir des phrases
documents = [text.split() for text in df['sentence_bow_lem']]
dictionary = corpora.Dictionary(documents)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in documents]

# Afficher le dictionnaire et la matrice document-termes
print("Dictionnaire :", dictionary)
for doc, bow in zip(documents[:5], doc_term_matrix[:5]):
    print(f"Phrase : {doc}")
    print(f"Bag of Words : {bow}")
    print()

['module named ipykernel launcher', 'using junit test pas command line argument spring boot application', 'what meaning const end member function declaration', 'react router dom typescript', 'get file size image width height upload', 'perform loop parallel python 3.2', 'write multiple output key spark one spark job', 'attempt insert non property list object trying save custom object swift', 'helper method model rail', 'how handle kendo grid row double click event', 'unit testing private service injected using jasmine angular2', 'changing image tag name kustomization.yaml', 'using eureka registry using rest apis', 'bootstrap fixed header footer scrolling body content area fluid container', 'swift replace substring regex', 'how install tensorflow python 2.7 window', 'object.keys using number typescript', 'color tint uibutton image', 'how efficiently parse csv file perl', 'vertical scrollbar frame tkinter python', 'ant warning includeantruntime set', 'subtract hour function', 'difference 

In [20]:
# LDA model
# chunksize détermine le nombre de documents à traiter en une seule fois (ou en un seul "chunk")
# passes indique le nombre de fois que l'algorithme doit passer sur l'ensemble complet des documents
# iterations détermine le nombre d'itérations à effectuer pour chaque passe.
lda = LdaModel(doc_term_matrix, num_topics=10, id2word = dictionary, random_state=0, chunksize=1000, passes=10, iterations=100)

# affiche les num_topics (10 ici) sujets les plus importants, en montrant les num_words (3 ici) mots les plus significatifs 
# pour chaque sujet. Cela donne un aperçu rapide des principaux thèmes détectés par le modèle  
print("3 mots les plus significatifs pour chaque topic:")
print(lda.print_topics(num_topics=10, num_words=3))
print("\n")

# Récupère les num_topics (10) sujets les plus importants, en montrant les num_words (10) mots les plus significatifs 
# pour chaque sujet. Le paramètre formatted=False renvoie les résultats sous forme de liste de tuples 
# plutôt que de chaîne de caractères formatée, ce qui est utile pour une manipulation ultérieure.
topics = lda.show_topics(num_topics=10, num_words=10, formatted=False)

# Pour chaque sujet, récupère l'index du sujet (topic_idx) et la liste des mots associés (topic)
# Chaque "Topic" affiché montre les mots les plus représentatifs pour ce sujet, 
# donnant un aperçu des thèmes principaux détectés par le modèle dans votre corpus de documents
for topic_idx, topic in topics:
    print(f"Topic {topic_idx}:")
    words = ", ".join([word for word, _ in topic])
    print(words)
    print()

3 mots les plus significatifs pour chaque topic:
[(0, '0.066*"how" + 0.026*"change" + 0.018*"text"'), (1, '0.025*"how" + 0.020*"variable" + 0.017*"django"'), (2, '0.057*"how" + 0.024*"spring" + 0.017*"using"'), (3, '0.017*"how" + 0.016*"row" + 0.014*"input"'), (4, '0.047*"how" + 0.037*"file" + 0.030*"error"'), (5, '0.039*"how" + 0.024*"python" + 0.019*"using"'), (6, '0.052*"what" + 0.029*"way" + 0.021*"difference"'), (7, '0.018*"name" + 0.016*"how" + 0.016*"why"'), (8, '0.028*"find" + 0.018*"working" + 0.014*"std"'), (9, '0.021*"java" + 0.020*"how" + 0.020*"parameter"')]


Topic 0:
how, change, text, image, io, android, swift, color, date, size

Topic 1:
how, variable, django, two, python, loop, check, ssl, table, exception

Topic 2:
how, spring, using, value, file, get, server, request, access, sql

Topic 3:
how, row, input, bootstrap, using, add, laravel, event, thread, getting

Topic 4:
how, file, error, use, using, line, command, studio, column, android

Topic 5:
how, python, using

In [21]:
# Initialiser une liste pour stocker les données
data = []

# Récupérer les termes et leurs probabilités pour chaque sujet
for topic_idx in range(lda.num_topics):
    topic_terms = lda.get_topic_terms(topicid=topic_idx, topn=10)  # topn peut être ajusté selon les besoins
    for term_id, prob in topic_terms:
        term = dictionary[term_id]
        data.append((topic_idx, term, prob))

# Créer un DataFrame à partir des données
df_lda = pd.DataFrame(data, columns=['Topic', 'Term', 'Probability'])

# Afficher le DataFrame
df_lda.head()

,Topic,Term,Probability
0,0,how,0.065866
1,0,change,0.026053
2,0,text,0.018071
3,0,image,0.017644
4,0,io,0.016948


In [22]:
# Supposons que `dictionary` est le dictionnaire de `gensim` créé à partir du corpus
# et que `corpus` est la collection de documents transformés en bag-of-words

# Récupérer les fréquences des termes
term_frequencies = [(term_id, freq) for term_id, freq in dictionary.dfs.items()]
term_frequencies = [(dictionary[term_id], freq) for term_id, freq in term_frequencies]

# Créer un DataFrame à partir des fréquences des termes
df_terms = pd.DataFrame(term_frequencies, columns=['Term', 'Frequency'])

# Trier le DataFrame par fréquence décroissante
df_terms_sorted = df_terms.sort_values(by='Frequency', ascending=False)

# Afficher le DataFrame des termes les plus fréquents
(df_terms_sorted.head(10))  # Afficher les 10 termes les plus fréquents

,Term,Frequency
58,how,1235
4,using,424
26,file,308
35,python,245
527,error,225
25,get,201
173,use,199
14,what,181
276,android,160
167,string,143


In [23]:
# Métriques
# calculate perplexity (densité de vraisemblance) : mesure la qualité de la modélisation d'un ensemble de données
print('\Perplexity: ', lda.log_perplexity(doc_term_matrix,
                                                total_docs=10000))  
 
# calculate topics coherence (mesure le degré de similitude sémantique entre les mots à score élevé dans un topic) : mesure de la qualité des topics générés par le modèle
# varie entre 0 et 1 : proche de 1 indique une haute cohérence
coherence_model_lda = CoherenceModel(model=lda,
                                     texts=documents, dictionary=dictionary, 
                                     coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence: ', coherence_lda)

\Perplexity:  -7.75082014161224


/Users/randaalsabbagh/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/randaalsabbagh/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/randaalsabbagh/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/randaalsabbagh/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, current

Coherence:  0.4225855216535492


In [24]:
# Visualisation du LDA avec 10 topics 
vis = gensimvis.prepare(lda, doc_term_matrix, dictionary)
pyLDAvis.display(vis)

### Conclusion: Les termes contenus dans chaque topic ne me permettent pas de spécifier un sujet et lui donner une étiquette. Le résultat Perplexity/Coherence n'est pas mauvais mais l'étiquettage étant le but du projet, ça ne m'aide pas.